In [1]:
from datetime import datetime
import os
import re
import time
from typing import LiteralString
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import json
import logging
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pyotp


def initialize_driver():
    now = datetime.now()
    local_now = now.astimezone()
    local_tz = local_now.tzinfo
    local_tzname = local_tz.tzname(local_now)
    print("current time ", datetime.now(), " local_tz  ", local_tzname)

    print("Initializing Selenium driver...")
    # print("Initializing Selenium driver...")
    options = Options()
    selenium_options = config["selenium_options"]

    # # Set headless option based on config
    if selenium_options["headless"]:
        options.add_argument("--headless")

    # Set window size
    options.add_argument(f'--window-size={selenium_options["window_size"]}')
    # Additional necessary arguments for headless mode in Docker
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")

    options.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    )

    # Test with profile
    # options.add_argument("user-data-dir=" + os.path.join(os.getcwd(), "ChromeProfile"))

    print("Driver initialized successfully.")

    return webdriver.Chrome(options=options)

def scrape_ch_mfa(driver):
    try:
        driver.execute_script("window.open('https://control.callharbor.com/portal/messages', '_blank');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.find_element(By.NAME, "data[Login][username]").send_keys("sms@proactivemgmt")
        driver.find_element(By.NAME, "data[Login][password]").send_keys("kB8zaXGbDD4-xdk0")
        driver.find_element(By.XPATH, '//input[@class="btn btn-large color-primary" and @type="submit" and @value="Log In"]').click()
        
        WebDriverWait(driver, 10).until(EC.url_contains("https://control.callharbor.com/portal/login/mfa/1"))
        
        mfa_code = pyotp.TOTP("JINDQR33AJDPJXED").now()
        driver.find_element(By.NAME, "data[Login][passcode]").send_keys(mfa_code)
        driver.find_element(By.XPATH, '//input[@class="btn btn-large color-primary" and @type="submit" and @value="Submit"]').click()
        
        driver.get("https://control.callharbor.com/portal/messages")
        WebDriverWait(driver, 10).until(EC.url_contains("https://control.callharbor.com/portal/messages"))
        
        message_xpath = '/html/body/div[2]/div[5]/div[2]/div[2]/div[1]/table/tbody/tr[1]/td[4]/div'
        message_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, message_xpath)))
        
        message_text = message_element.text
        match = re.search(r"Your code is: (\d+)", message_text)
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        return match.group(1) if match else None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def handle_mfa(driver):
    print("Handling MFA...")
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "sendCallButton"))
    )

    send_call_button = driver.find_element(By.ID, "sendCallButton")
    send_call_button.click()
    time.sleep(5)
    code = scrape_ch_mfa(driver)
    if not code:
        raise Exception("MFA code not retrieved.")

    code_field = driver.find_element(By.ID, "code")
    code_field.send_keys(code)

    send_code_button = driver.find_element(By.ID, "sendCodeButton")
    send_code_button.click()

    # Wait for the MFA process to complete and navigate to the next page
    WebDriverWait(driver, 20).until(EC.url_changes)
    print("MFA handled successfully.")


def login(driver, username, password, url):
    try:

        print(f"Attempting login for user: {username}")
        # print(f"Attempting login for user: {username}")
        driver.get(url)

        username_field = driver.find_element(By.ID, "inputUsername")
        username_field.clear()
        username_field.send_keys(username)

        password_field = driver.find_element(By.ID, "inputPswd")
        password_field.send_keys(password)
        login_button = driver.find_element(By.ID, "loginButton")
        login_button.click()

        # Handle MFA
        if config.get("mfa"):
            handle_mfa(driver)

    except Exception as e:
        print(f"An exception occurred during login: {e}")

        # print(f"An exception occurred during login: {e}")
        raise e

    # def GetRecods(driver, record):


def accept_alert(driver):
    try:
        WebDriverWait(driver, 10).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert.accept()
    except:
        print(" accept_alert ")
        pass


def get_appointments(driver):
    schedule_url: LiteralString = (
        "https://static.practicefusion.com/apps/ehr/index.html?utm_source=exacttarget&utm_medium=email&utm_campaign=InitialSetupWelcomeAddedUser#/PF/schedule/scheduler/agenda"
    )
    driver.get(schedule_url)
    # Call this function before interacting with elements that might trigger alerts
    accept_alert(driver)
    time.sleep(10)
    print(f"driver.current_url: {driver.current_url}")

    if config.get("get_yestdays_records"):
        decrementbutton = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[@class='btn-sm decrement-date']")
            )
        )
        if decrementbutton:
            decrementbutton.click()
            print("decrementbutton clicked ")
        else:
            print("decrementbutton not found ")

        time.sleep(10)

    # Wait until the button is clickable
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (
                By.XPATH,
                "//button[@class='btn--default' and @data-element='btn-schedule-print']",
            )
        )
    )
    print("schedulebutton clicked ")
    button.click()

    time.sleep(3)
    # Find the HTML element representing the table
    table = driver.find_element(
        By.CSS_SELECTOR, 'table[data-element="table-agenda-print"]'
    )

    # Get all the rows in the table
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows (skipping the header row) and extract the data
    all_row_data = []
    for row in rows[1:]:
        cols = row.find_elements(By.TAG_NAME, "td")
        row_data = []
        for col in cols:
            cell_text = col.get_attribute("textContent").strip()
            contact_details = col.find_elements(By.CSS_SELECTOR, "div.contact-details")
            if contact_details:
                details = [
                    detail.text.strip()
                    for detail in contact_details[0].find_elements(By.TAG_NAME, "div")
                ]
                cell_text += f" ({', '.join(details)})"
            row_data.append(cell_text)
        all_row_data.append(row_data)

    appointments = []
    for row in all_row_data:
        phone_number = row[1].split("\n")[8].split("(, , )")[0].strip()
        patientPhone = "".join(filter(str.isdigit, phone_number))
        time_object = datetime.strptime(row[2], "%I:%M %p").time()
        # # Create a dummy date (e.g., today's date)
        dummy_date = datetime.today().date()
        # # Combine the time object with the dummy date
        appointmentTime = datetime.combine(dummy_date, time_object)
        patientDOBraw = row[1].split("\n")[3].strip()
        # Convert the input date string to a datetime object
        date_object: datetime = datetime.strptime(patientDOBraw, "%m/%d/%Y")
        patientDOB = date_object.strftime("%Y-%m-%d")

        appointment = {
            "patientName": row[1].split("\n")[0].strip(),
            "patientDOB": patientDOB,
            "patientPhone": patientPhone,
            "appointmentTime": appointmentTime.strftime("%Y-%m-%dT%H:%M"),
            "appointmentStatus": row[0],
            "provider": row[3],
            "type": row[4],
        }
        appointments.append(appointment)

    # Convert the list of dictionaries to JSON

    return json.dumps(appointments, indent=4)


with open("config.json") as config_file:
    config = json.load(config_file)
    credentials = config["credentials"]
    username = credentials["username"]
    password = credentials["password"]
    login_url = credentials["login_url"]


def return_appointments():
    try:
        driver = initialize_driver()
        if driver:
            print(" driver Succesfully Initialized...")

        login(driver, username, password, login_url)
        time.sleep(2)
        # print("Starting to process accounts...")
        appointments = get_appointments(driver)

        return appointments

    except Exception as e:
        raise e


def run_rpa():
    try:
        print("First attempt. Appointments: ")
        return return_appointments()
    except:

        print("Fist attempt failed. Waiting 60 second to second attempt... ")
        time.sleep(60)
        return return_appointments()


if __name__ == "__main__":
    run_rpa()

First attempt. Appointments: 
current time  2024-07-17 16:03:33.543492  local_tz   Eastern Daylight Time
Initializing Selenium driver...
Driver initialized successfully.
 driver Succesfully Initialized...
Attempting login for user: syoung@commongroundhelps.org
Handling MFA...
MFA handled successfully.
driver.current_url: https://static.practicefusion.com/apps/ehr/index.html?utm_source=exacttarget&utm_medium=email&utm_campaign=InitialSetupWelcomeAddedUser#/PF/schedule/scheduler/agenda
schedulebutton clicked 


In [29]:
def scrape_ch_mfa(driver):
    try:
        driver.get("https://control.callharbor.com/portal/messages")
        driver.find_element(By.NAME, "data[Login][username]").send_keys("sms@proactivemgmt")
        driver.find_element(By.NAME, "data[Login][password]").send_keys("kB8zaXGbDD4-xdk0")
        driver.find_element(By.XPATH, '//input[@class="btn btn-large color-primary" and @type="submit" and @value="Log In"]').click()
        
        WebDriverWait(driver, 10).until(EC.url_contains("https://control.callharbor.com/portal/login/mfa/1"))
        
        mfa_code = pyotp.TOTP("JINDQR33AJDPJXED").now()
        driver.find_element(By.NAME, "data[Login][passcode]").send_keys(mfa_code)
        driver.find_element(By.XPATH, '//input[@class="btn btn-large color-primary" and @type="submit" and @value="Submit"]').click()
        
        driver.get("https://control.callharbor.com/portal/messages")
        WebDriverWait(driver, 10).until(EC.url_contains("https://control.callharbor.com/portal/messages"))
        
        message_xpath = '/html/body/div[2]/div[5]/div[2]/div[2]/div[1]/table/tbody/tr[1]/td[4]/div'
        message_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, message_xpath)))
        
        message_text = message_element.text
        match = re.search(r"Your code is: (\d+)", message_text)
        return match.group(1) if match else None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# driver = initialize_driver()
# scrape_ch_mfa(driver)

current time  2024-07-17 15:56:43.760574  local_tz   Eastern Daylight Time
Initializing Selenium driver...
Driver initialized successfully.


'31830'

In [27]:
print(username)
print(password)
print(login_url)

initialize_driver()
return_appointments()

syoung@commongroundhelps.org
Evolution!
https://static.practicefusion.com/apps/ehr/index.html#/login
current time  2024-07-17 15:53:49.024135  local_tz   Eastern Daylight Time
Initializing Selenium driver...
Driver initialized successfully.
current time  2024-07-17 15:53:52.347114  local_tz   Eastern Daylight Time
Initializing Selenium driver...
Driver initialized successfully.
 driver Succesfully Initialized...
Attempting login for user: syoung@commongroundhelps.org
Handling MFA...
Requesting code...
access_token: 129dce4de08da1e40e5136a33cd41534
Error: 401 - 
Error: No message
An exception occurred during login: MFA code not retrieved.


Exception: MFA code not retrieved.

In [ ]:
if __name__ == "__main__":
    run_rpa()
